In [ ]:
import json
import os
import pandas as pd
import pyshark
import random
import re
from collections import OrderedDict
from json import JSONDecoder
from json import loads
from json import dumps
from os.path import isfile, join
from os import listdir
from pprint import pprint
from subprocess import Popen, PIPE, check_output

# Extract funtion is used to flatten nested dictionaries to a Lvl 1 Dictionary
# Cons: It does not avoid collisions
def extract(dict_in, dict_out):
    for key, value in dict_in.iteritems():
        if isinstance(value, dict): # If value itself is dictionary
            extract(value, dict_out)
        elif isinstance(value, unicode):
            # Write to dict_out
            dict_out[key] = value
    return dict_out

# Parse Object Pairs function identifies all keys that are equal at the same level
# and calls make_unique to rename the keyname
def parse_object_pairs(pairs):
    dct = OrderedDict()
    #dct = dict()
    for key, value in pairs:
        if key in dct:
            key = make_unique(key, dct)
        dct[key] = value
    return dct

# Make Unique function is in charge of renaming keys which have the same name 
def make_unique(key, dct):
    counter = 0
    unique_key = key

    while unique_key in dct:
        counter += 1
        unique_key = '{}_{}'.format(key, counter)
    return unique_key

# Flatten DIct function is in charge of flattening the nested dictionaries
# While flattening, it will rename the keys at the last level by prepending predecesor key names
def flatten_dict(d, prefix='_'):
    def items():
        # A clojure for recursively extracting dict like values
        for key, value in d.items():
            if isinstance(value, dict):
                for sub_key, sub_value in flatten_dict(value).items():
                    # Key name should imply nested origin of the dict,
                    # so we use a default prefix of __ instead of _ or .
                    if key == "_source" or key == "layers":
                        yield sub_key, sub_value
                    else:
                        yield key + prefix + sub_key, sub_value
            else:
                yield key, value
    return dict(items())

# Define the Main Directory containing all uncompressed PCAPS
pcaps_dir='/home/cc/Contagio'
# Create a list of the Sub-directories within the Main Directory
directories = [x[0] for x in os.walk(pcaps_dir)]

# Print list of Sub-directories
print(directories)
uncpx_pcaps_dir = directories

# Define required empty lists
failed_pcaps=[]

# Define variables
mw_id_counter = 0

# Create List of Unique IDs to identify to assign to each Malware PCAP
mw_random_ids = sorted(random.sample(range(10000,20000), 1300))
#mw_id = mw_random_ids[0]

# Dictionary for matching Malware ID and Malware Name
mw_id_dict = {}

#Create Empty Dataframe
columns = ['mw_id']
df_ = pd.DataFrame(index=[0], columns=columns)

# The following loop will perform the following actions:
# 1) Loop through every Sub-directory
# 2) Loop through every PCAP within each Sub-directory
# 3) Read every PCAP using tshark filtering only frames containing ssl traffic
#    In addtion, the output of all protocol features will be delivered in a nested JSON format
# 4) The extracted dictionary will be parsed using parse_object_pairs and flatten_dict function
for pcap_dir in uncpx_pcaps_dir[0:1]: #uncpx_pcaps_dir[0:1]
    pcap_dir = "/home/cc/Contagio/Exploit_Kits_and_Malware"
    print("\n")
    print(pcap_dir)
    pcap_dir_files = [f for f in listdir(pcap_dir) if isfile(join(pcap_dir, f))]
    print("\n")
    #pcap_dir_files = ["EK_MALWARE_2014-04-29-fake-flash-updater-02_mailware-traffic-analysis.net.pcap"]
    for i_cap in pcap_dir_files:
        # Get a new malware ID
        mw_id = mw_random_ids[mw_id_counter]
        # Save Malware Id and Name to Dictionary
        mw_id_dict[mw_id] = i_cap
        print("ID: %s" %mw_id)
        print("MW: %s" %mw_id_dict[mw_id])
        # Increase Counter
        mw_id_counter = mw_id_counter + 1
        # Print PCAP
        print(i_cap)
        # Executed Tshark Commands to Filter frames containing SSL in PCAP
        command = ["tshark", "-n", "-r", "{}".format(pcap_dir+'/'+i_cap), "-Y", "ssl", "-T", "fields", "-e", "frame.number"]
        #command = ["tshark", "-n", "-r", "{}".format(pcap_dir+'/'+i_cap), "-Y", "ssl", "-T", "json"]
        #command = ["tshark", "-n", "-r", "{}".format(pcap_dir+'/'+i_cap), "-2", "-R", "ssl", "-Y", "frame.number==1", "-T", "json" ]
        process = Popen(command, universal_newlines=True, stdout=PIPE, stderr=PIPE)
        output, err = process.communicate(b"input data that is passed to subprocess' stdin")
        rc = process.returncode
        # Print Number of Packets in SSL Frame
        #packet_numbers = [int(x) for x in output[:-1].split('\n')]
        packet_numbers = list(map(int, re.findall(r'\d+', output)))
        print("Number of Frames containing SSL: %s" %len(packet_numbers))
        print("\n")
        # Loop for extracting and parsing data per frame on PCAP
        for p_num in packet_numbers:
            command = ["tshark", "-n", "-r", "{}".format(pcap_dir+'/'+i_cap), "-Y", "frame.number=={}".format(p_num), "-T", "json" ]
            process2 = Popen(command, universal_newlines=True, stdout=PIPE, stderr=PIPE)
            output2, err2 = process2.communicate(b"input data that is passed to subprocess' stdin")
            decoder = JSONDecoder(object_pairs_hook=parse_object_pairs)
            user_dict = decoder.decode(output2[2:-4])
            # Flatten received dictionary
            user_dict_final = flatten_dict(user_dict)
            # Add Malware ID to Temp Dataframe
            user_dict_final["mw_id"] = mw_id
            # Create Temp Dataframe
            df = pd.DataFrame(user_dict_final, index=[1])
            # Append Temp Dataframe to Main Dataframe (df_)
            #df_ = pd.concat([df_,df])
            df_ = df_.append([df], ignore_index=True)

print("\nFailed PCAPS")
print(failed_pcaps)


['/home/cc/Contagio', '/home/cc/Contagio/Exploit_Kits_and_Malware', '/home/cc/Contagio/Exploit_Kits_and_Malware/EK_MALWARE_2014-08-13-sandbox-analysis-pcap-files_mailware-traffic-analysis.net', '/home/cc/Contagio/Exploit_Kits_and_Malware/EK_MALWARE_2015-01-08-pcap-files-associated-with-malware-from-82.244.160.22_mailware-traffic-analysis.net', '/home/cc/Contagio/Exploits_and_Scans', '/home/cc/Contagio/APT', '/home/cc/Contagio/GENERAL_DEFCON_17', '/home/cc/Contagio/General', '/home/cc/Contagio/CRIME', '/home/cc/Contagio/CRIME/tbotpcaps']


/home/cc/Contagio/Exploit_Kits_and_Malware


ID: 10006
MW: EK_MALWARE_2014-03-09-callback-traffic-from-physical-host_mailware-traffic-analysis.net.pcap
EK_MALWARE_2014-03-09-callback-traffic-from-physical-host_mailware-traffic-analysis.net.pcap
Number of Frames containing SSL: 0


ID: 10022
MW: EK_MALWARE_2014-08-29-phishing-malware-01-sandbox-analysis_mailware-traffic-analysis.net.pcap
EK_MALWARE_2014-08-29-phishing-malware-01-sandbox-analysis_mailwa

In [ ]:
# Create list of discovered features
total_features = list(df_.columns.values)
print(len(total_features))
#pprint(total_features)


In [ ]:
# Create mw_id_series
mw_id_series = pd.Series(mw_id_dict)
# Save mw_id_series
mw_id_series.to_pickle('mw_id_series.p')
# Save SSL df (df_)
df_.to_pickle('mw_ssl_dataframe.p')
# Save SSL features list
ssl_features_file = open('ssl_features.txt', 'w')
for item in total_features:
    ssl_features_file.write("%s\n" % item)

# Sources:

## Tshark
https://ask.wireshark.org/questions/58343/2-pass-filter-in-wiresharktshark
https://hackertarget.com/tshark-tutorial-and-filter-examples/
https://www.wireshark.org/docs/wsug_html_chunked/AppToolstshark.html
http://www.thegeekstuff.com/2012/07/wireshark-filter/?ref=driverlayer

## Pandas and JSON
https://www.dataquest.io/blog/python-json-tutorial/
https://stackoverflow.com/questions/21104592/json-to-pandas-dataframe
https://pandas.pydata.org/pandas-docs/stable/generated/pandas.io.json.json_normalize.html
https://pandas.pydata.org/pandas-docs/stable/merging.html
https://stackoverflow.com/questions/23019119/converting-multilevel-nested-dictionaries-to-pandas-dataframe
http://pbpython.com/pandas-list-dict.html
https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html
https://stackoverflow.com/questions/13784192/creating-an-empty-pandas-dataframe-then-filling-it
https://pandas.pydata.org/pandas-docs/stable/options.html
http://www.gregreda.com/2013/10/26/working-with-pandas-dataframes/
https://www.digitalocean.com/community/tutorials/an-introduction-to-the-pandas-package-and-its-data-structures-in-python-3

## Unique Dictionary Keys
https://stackoverflow.com/questions/29321677/python-json-parser-allow-duplicate-keys
https://stackoverflow.com/questions/36086470/parsing-python-json-with-multiple-same-strings-with-different-values
https://stackoverflow.com/questions/29321677/python-json-parser-allow-duplicate-keys/29323197

## Flattening Dictionaries
https://stackoverflow.com/questions/12507206/python-recommended-way-to-walk-complex-dictionary-structures-imported-from-json
https://stackoverflow.com/questions/6027558/flatten-nested-python-dictionaries-compressing-keys

## Zip Files
https://unix.stackexchange.com/questions/270555/how-to-get-number-of-files-in-a-7z